# Analysis of the filtered dataset

Banter

In [1]:
import pandas as pd
import numpy as np
import math
from bson import ObjectId
from pymongo import MongoClient
from folium import Map, Marker, Icon, FeatureGroup, LayerControl, Choropleth
from folium.plugins import HeatMap
from folium.vector_layers import Circle, Polygon
from sklearn.metrics.pairwise import haversine_distances
from math import radians
from functions.analysis import *
import re
import os
import requests
from dotenv import load_dotenv
import json

client = MongoClient()

load_dotenv()
id_4sq = os.getenv("FOURSQ_ID")
tk_4sq = os.getenv("FOURSQ_SEC")

In [2]:
candidates = pd.read_csv("temporal_database/candidates_airports_def.csv", index_col=0, encoding = "ISO-8859-1")

In [3]:
candidates.head()

,level_0,index,_id,category_code,id_near,name_near,address1,address2,city,latitude,...,num_night,has_daycare,name_daycare,coord_daycare,has_pet,name_pet,coord_pet,has_airport,name_airport,coord_airpott
0,0,0,5faf9c3760efce486b0d5450,network_hosting,5faf9c3660efce486b0d4e7c,Wetpaint,615 2nd Avenue Suite 280,NaN,Seattle,47.602416,...,[20],[True],['Bright Horizons at Fourth and Madison'],"[-122.3336027, 47.6054777]",[True],['Doney Memorial Pet Clinic'],"[-122.330604, 47.601537]",[True],['Seattle-Tacoma International Airport (SEA) (...,"[-122.302508354187, 47.44358853419229]"
1,1,1,5faf9c3860efce486b0d6666,web,5faf9c3660efce486b0d4e7c,Wetpaint,615 Second Avenue,Suite 280,Seattle,47.602416,...,[20],[True],['Bright Horizons at Fourth and Madison'],"[-122.3336027, 47.6054777]",[True],['Doney Memorial Pet Clinic'],"[-122.330604, 47.601537]",[True],['Seattle-Tacoma International Airport (SEA) (...,"[-122.302508354187, 47.44358853419229]"
2,2,2,5faf9c3860efce486b0d7075,software,5faf9c3660efce486b0d4e7c,Wetpaint,2200 Alaskan Way,Suite 130,Seattle,47.603873,...,[20],[True],['Bright Horizons at Fourth and Madison'],"[-122.3336027, 47.6054777]",[True],['Doney Memorial Pet Clinic'],"[-122.330604, 47.601537]",[True],['Seattle-Tacoma International Airport (SEA) (...,"[-122.302508354187, 47.44358853419229]"
3,3,3,5faf9c3860efce486b0d694a,software,5faf9c3660efce486b0d4e7c,Wetpaint,810 3rd Ave Ste 242,NaN,Seattle,47.604266,...,[20],[True],['Bright Horizons at Fourth and Madison'],"[-122.3336027, 47.6054777]",[True],['Doney Memorial Pet Clinic'],"[-122.330604, 47.601537]",[True],['Seattle-Tacoma International Airport (SEA) (...,"[-122.302508354187, 47.44358853419229]"
4,5,5,5faf9c3760efce486b0d5150,web,5faf9c3660efce486b0d4e7c,Wetpaint,811 First Avenue,Suite 480,Seattle,47.603327,...,[20],[True],['Bright Horizons at Fourth and Madison'],"[-122.3336027, 47.6054777]",[True],['Doney Memorial Pet Clinic'],"[-122.330604, 47.601537]",[True],['Seattle-Tacoma International Airport (SEA) (...,"[-122.302508354187, 47.44358853419229]"


In [4]:
candidates.columns

Index(['level_0', 'index', '_id', 'category_code', 'id_near', 'name_near',
       'address1', 'address2', 'city', 'latitude', 'longitude', 'coords',
       'has_veg', 'name_veg', 'coord_veg', 'num_veg', 'has_basket',
       'name_basket', 'coord_basket', 'has_night', 'name_night', 'coord_night',
       'num_night', 'has_daycare', 'name_daycare', 'coord_daycare', 'has_pet',
       'name_pet', 'coord_pet', 'has_airport', 'name_airport',
       'coord_airpott'],
      dtype='object')

In [5]:
col_unfold = [ 'has_veg', 'name_veg', 'coord_veg', 'num_veg', 'has_basket',
       'name_basket', 'coord_basket', 'has_night', 'name_night', 'coord_night',
       'num_night', 'has_daycare', 'name_daycare', 'coord_daycare', 'has_pet',
       'name_pet', 'coord_pet', 'coord_pet', 'has_airport', 'name_airport',
       'coord_airpott']
integer_cols = ['num_night','num_veg']
longlat =['latitude', 'longitude']

col_coords = [ 'coord_veg', 'coord_basket', 'coord_night', 'coord_daycare', 'coord_pet', 'coord_airpott']
bolcols = [ 'has_veg', 'has_basket','has_night','has_daycare','has_pet', 'has_airport']

for col in col_unfold:
    candidates[col]=candidates[col].apply(lambda x: x[1:-1])
    
for col in integer_cols:
    candidates[col]=candidates[col].apply(int)

for col in longlat:
    candidates[col]=candidates[col].apply(float) 

for col in bolcols:
    candidates[col]=candidates[col].apply(lambda x: x == "True")  

def change_coords(x):    
    xc=re.split(r'(,)', x)
    if not("None" in xc) and not("one" in xc):
        
        return [float(xc[-1]),float(xc[0])]
    else:
        return None

for col in col_coords:
    #candidates[col]=candidates[col].apply(lambda x: [float(xc) for xc in x.split(",") if (x[:4] != "None")])     
    candidates[col]=candidates[col].apply(change_coords) 
    
    

In [6]:
candidates.head()

,level_0,index,_id,category_code,id_near,name_near,address1,address2,city,latitude,...,num_night,has_daycare,name_daycare,coord_daycare,has_pet,name_pet,coord_pet,has_airport,name_airport,coord_airpott
0,0,0,5faf9c3760efce486b0d5450,network_hosting,5faf9c3660efce486b0d4e7c,Wetpaint,615 2nd Avenue Suite 280,NaN,Seattle,47.602416,...,20,True,'Bright Horizons at Fourth and Madison',"[47.6054777, -122.3336027]",True,'Doney Memorial Pet Clinic',"[47.60153, 122.330604]",True,'Seattle-Tacoma International Airport (SEA) (S...,"[47.44358853419229, -122.302508354187]"
1,1,1,5faf9c3860efce486b0d6666,web,5faf9c3660efce486b0d4e7c,Wetpaint,615 Second Avenue,Suite 280,Seattle,47.602416,...,20,True,'Bright Horizons at Fourth and Madison',"[47.6054777, -122.3336027]",True,'Doney Memorial Pet Clinic',"[47.60153, 122.330604]",True,'Seattle-Tacoma International Airport (SEA) (S...,"[47.44358853419229, -122.302508354187]"
2,2,2,5faf9c3860efce486b0d7075,software,5faf9c3660efce486b0d4e7c,Wetpaint,2200 Alaskan Way,Suite 130,Seattle,47.603873,...,20,True,'Bright Horizons at Fourth and Madison',"[47.6054777, -122.3336027]",True,'Doney Memorial Pet Clinic',"[47.60153, 122.330604]",True,'Seattle-Tacoma International Airport (SEA) (S...,"[47.44358853419229, -122.302508354187]"
3,3,3,5faf9c3860efce486b0d694a,software,5faf9c3660efce486b0d4e7c,Wetpaint,810 3rd Ave Ste 242,NaN,Seattle,47.604266,...,20,True,'Bright Horizons at Fourth and Madison',"[47.6054777, -122.3336027]",True,'Doney Memorial Pet Clinic',"[47.60153, 122.330604]",True,'Seattle-Tacoma International Airport (SEA) (S...,"[47.44358853419229, -122.302508354187]"
4,5,5,5faf9c3760efce486b0d5150,web,5faf9c3660efce486b0d4e7c,Wetpaint,811 First Avenue,Suite 480,Seattle,47.603327,...,20,True,'Bright Horizons at Fourth and Madison',"[47.6054777, -122.3336027]",True,'Doney Memorial Pet Clinic',"[47.60153, 122.330604]",True,'Seattle-Tacoma International Airport (SEA) (S...,"[47.44358853419229, -122.302508354187]"


In [7]:
candidates.columns

Index(['level_0', 'index', '_id', 'category_code', 'id_near', 'name_near',
       'address1', 'address2', 'city', 'latitude', 'longitude', 'coords',
       'has_veg', 'name_veg', 'coord_veg', 'num_veg', 'has_basket',
       'name_basket', 'coord_basket', 'has_night', 'name_night', 'coord_night',
       'num_night', 'has_daycare', 'name_daycare', 'coord_daycare', 'has_pet',
       'name_pet', 'coord_pet', 'has_airport', 'name_airport',
       'coord_airpott'],
      dtype='object')

In [8]:
obtain_dist([candidates["latitude"][0], candidates["longitude"][0]] ,candidates["coord_daycare"][0])

348.4219125653339

In [9]:
def obtain_score(df):
    n=len(df)
    score = np.ones(n)
    
    for i in range(n):
        #Airport ckeck
        if df["has_airport"][i]:
            score[i] += 1.
        #Vegan check
        if df["has_veg"][i]:
            score[i] += ( df["num_veg"][i] / 10) * 0.5
            dist = obtain_dist([candidates["latitude"][i], candidates["longitude"][i]]
                                    ,candidates["coord_veg"][i])
            score[i] += sigmoid(dist,0.25,100) * 0.5
        #Child daycare check
        if df["has_daycare"][i]:
            dist = obtain_dist([candidates["latitude"][i], candidates["longitude"][i]]
                                    ,candidates["coord_daycare"][i])
            score[i] += sigmoid(dist,0.25,250) 
        #Basket stadium check
        if df["has_basket"][i]:
            score[i] += 1.
        #Nightlife check
        if df["has_night"][i]:
            score[i] += (df["num_night"][i] / 20) *0.5  
            dist = obtain_dist([candidates["latitude"][i], candidates["longitude"][i]]
                                    ,candidates["coord_night"][i])
            score[i] += sigmoid(dist,0.25,100) * 0.5 
        #Child daycare check
        if df["has_pet"][i]:
            dist = obtain_dist([candidates["latitude"][i], candidates["longitude"][i]]
                                    ,candidates["coord_pet"][i])
            score[i] += sigmoid(dist,0.25,250) 
            
    
    return pd.Series(score)

In [10]:
candidates["score"] = obtain_score(candidates)

/home/ordovas/IRONHACK/mini-projects/geospatial-data-project/functions/analysis.py:25: RuntimeWarning: overflow encountered in exp
  return 1./(1. + np.exp(-c1 * (c2 - d)))


In [11]:
candidates=candidates.sort_values(by=['score'], ascending=False)
candidates[:3]

,level_0,index,_id,category_code,id_near,name_near,address1,address2,city,latitude,...,has_daycare,name_daycare,coord_daycare,has_pet,name_pet,coord_pet,has_airport,name_airport,coord_airpott,score
1176,4722,4722,5faf9c3860efce486b0d6620,software,5faf9c3760efce486b0d4fc5,SelectMinds,"246, Fifth Avenue",Ste. 301,New York,40.744549,...,True,'Apple Seeds',"[40.743248277718095, -73.98961352390818]",True,'New York Dog Spa & Hotel',"[40.74370946621388, 73.99053168978655]",True,'Teterboro Airport (TEB) (Teterboro Airport)',"[40.85568601427126, -74.06060052691976]",5.999973
334,436,436,5faf9c3760efce486b0d5c09,other,5faf9c3660efce486b0d4e9f,Kyte,116 Natoma St.,3rd Floor,San Francisco,37.787076,...,True,'Kids by the Bay - Financial District',"[37.785471, -122.39803700000002]",True,'Pawtrero Bathhouse & Feed Co.',"[37.7838, 122.3893422]",True,'San Francisco International Airport (SFO) (Sa...,"[37.6167130000997, -122.38709449768066]",5.999427
1179,4726,4726,5faf9c3860efce486b0d6fec,ecommerce,5faf9c3760efce486b0d4fc5,SelectMinds,244 5th Avenue,Suite 2LP,New York,40.744618,...,True,'Apple Seeds',"[40.743248277718095, -73.98961352390818]",True,'New York Dog Spa & Hotel',"[40.74370946621388, 73.99053168978655]",True,'Teterboro Airport (TEB) (Teterboro Airport)',"[40.85568601427126, -74.06060052691976]",5.998199


In [12]:
long=candidates[candidates["_id"] == "5faf9c3860efce486b0d6620"]["longitude"]
lat=candidates[candidates["_id"] == "5faf9c3860efce486b0d6620"]["latitude"]

In [13]:
m = Map(location=[lat, long],zoom_start=15 )
""" tiles parameter
tiles = cartodbpositron
        stamenterrain
        stamentoner
""";

In [14]:
url = 'https://api.foursquare.com/v2/venues/explore'

ic = Icon(color="red",prefix="fa", icon="gamepad")
selected = Marker(location=[lat, long], tooltip="CHOSEN LOCATION", popup="Generic Gaming Company S.L.",icon=ic)
selected.add_to(m)


long_near=obtain_closer_data(client,"5faf9c3760efce486b0d4fc5")[0]['offices']['longitude']
lat_near=obtain_closer_data(client,"5faf9c3760efce486b0d4fc5")[0]['offices']['latitude']
name_near=obtain_closer_data(client,"5faf9c3760efce486b0d4fc5")[0]['name']
ic = Icon(color="blue",prefix="fa", icon="gear")
mrk=Marker(location=[lat_near, long_near], tooltip="Neighbour Succesful Tech Company", popup=name_near,icon=ic)
mrk.add_to(m)


lat_t=candidates[candidates["_id"] == "5faf9c3860efce486b0d6620"]["coord_airpott"].values[0][0]
long_t=candidates[candidates["_id"] == "5faf9c3860efce486b0d6620"]["coord_airpott"].values[0][1]
name_t=candidates[candidates["_id"] == "5faf9c3860efce486b0d6620"]["name_airport"].values[0]
ic = Icon(color="gray",prefix="fa", icon="plane")
mrk=Marker(location=[lat_t, long_t], tooltip="Airport", popup=name_t,icon=ic)
mrk.add_to(m)


lat_t=candidates[candidates["_id"] == "5faf9c3860efce486b0d6620"]["coord_basket"].values[0][0]
long_t=candidates[candidates["_id"] == "5faf9c3860efce486b0d6620"]["coord_basket"].values[0][1]
name_t=candidates[candidates["_id"] == "5faf9c3860efce486b0d6620"]["name_basket"].values[0]
ic = Icon(color="orange",prefix="fa", icon="dribbble")
mrk=Marker(location=[lat_t, long_t], tooltip="Basketball Stadium", popup=name_t,icon=ic)
mrk.add_to(m)


lat_t=candidates[candidates["_id"] == "5faf9c3860efce486b0d6620"]["coord_daycare"].values[0][0]
long_t=candidates[candidates["_id"] == "5faf9c3860efce486b0d6620"]["coord_daycare"].values[0][1]
name_t=candidates[candidates["_id"] == "5faf9c3860efce486b0d6620"]["name_daycare"].values[0]
ic = Icon(color="pink",prefix="fa", icon="user")
mrk=Marker(location=[lat_t, long_t], tooltip="Child Daycare", popup=name_t,icon=ic)
mrk.add_to(m)


lat_t=candidates[candidates["_id"] == "5faf9c3860efce486b0d6620"]["coord_pet"].values[0][0]
long_t=candidates[candidates["_id"] == "5faf9c3860efce486b0d6620"]["coord_pet"].values[0][1]
name_t=candidates[candidates["_id"] == "5faf9c3860efce486b0d6620"]["name_pet"].values[0]
ic = Icon(color="purple",prefix="fa", icon="paw")
mrk=Marker(location=[lat_t, -long_t], tooltip="Per Services", popup=name_t,icon=ic)
mrk.add_to(m)



params = dict(client_id=id_4sq,client_secret=tk_4sq,v='20201114',
        ll=str(lat.values[0])+","+str(long.values[0]),limit=20,
        categoryId="4d4b7105d754a06376d81259",radius=2000)
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)
print(data.keys)
for i in range(len(data["response"]["groups"][0]["items"])):
    long_t = data["response"]["groups"][0]["items"][i]["venue"]["location"]["lng"]
    lat_t = data["response"]["groups"][0]["items"][i]["venue"]["location"]["lat"]
    name_t = data["response"]["groups"][0]["items"][i]["venue"]["name"]
    ic = Icon(color="black",prefix="fa", icon="beer")
    mrk=Marker(location=[lat_t, long_t], tooltip="Nightlife",
               popup=name_t,icon=ic)
    mrk.add_to(m)
    

params = dict(client_id=id_4sq,client_secret=tk_4sq,v='20201114',
        ll=str(lat.values[0])+","+str(long.values[0]),limit=10,
        categoryId="4bf58dd8d48988d1d3941735",radius=500)
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)
print(data.keys)
for i in range(len(data["response"]["groups"][0]["items"])):
    long_t = data["response"]["groups"][0]["items"][i]["venue"]["location"]["lng"]
    lat_t = data["response"]["groups"][0]["items"][i]["venue"]["location"]["lat"]
    name_t = data["response"]["groups"][0]["items"][i]["venue"]["name"]
    ic = Icon(color="green",prefix="fa", icon="leaf")
    mrk=Marker(location=[lat_t, long_t], tooltip="Vegan Spots",
               popup=name_t,icon=ic)
    mrk.add_to(m)
        
    
m

<built-in method keys of dict object at 0x7fd962050cc0>
<built-in method keys of dict object at 0x7fd962065380>


In [17]:
m2 = Map(location=[lat, long],zoom_start=1 )

for i in range(len(candidates)):
    if candidates["score"][i] > candidates["score"].describe()["75%"]:
        lat_t=candidates["latitude"][i]
        long_t=candidates["longitude"][i]
        name_t="Score: "+str(candidates["score"][i])
        ic = Icon(color="red",prefix="fa", icon="rebel")
        mrk=Marker(location=[lat_t, long_t], tooltip="Candidate", popup=name_t,icon=ic)
        mrk.add_to(m2)



m2

In [16]:
i=0
candidates["latitude"][i]

47.602416